# Sentiment Analysis with Large Language Model embeddings

## Pre-requisites
- Setup a LiteLLM proxy instance (as mentioned at `.env.sample` file) to call LLM API embeddings

## Our Dataset

This dataset describes the contents of the heart-disease diagnosis.

The dataset in this study is from [Kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data), which is called Twitter US Airline Sentiment.

- Dataset: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data

## Variable Table

| Original Dataset             | Data Type     | Description    |                                                         
|------------------------------|---------------|---------------------------------------------------------------------------------------------|
| tweet_id                     | ID            | A unique identifier for each tweet.                                                         | 
| airline_sentiment            | Categorical   | The sentiment expressed in the tweet (positive, neutral, negative).                         | 
| airline_sentiment_confidence | Numerical     | Confidence score in the sentiment label (0 to 1).                                           | 
| negativereason               | Categorical   | Reason for negative sentiment (e.g., "Late Flight", "Customer Service Issue").              | 
| negativereason_confidence    | Numerical     | Confidence score in the negative reason label (0 to 1).                                     | 
| airline                      | Categorical   | The airline mentioned in the tweet (e.g., United, Delta, etc.).                             | 
| airline_sentiment_gold       | Categorical   | Sentiment label by trusted annotator (gold standard).                                       | 
| name                         | Text          | Name of the user who posted the tweet.                                                      | 
| negativereason_gold          | Categorical   | Negative reason label by trusted annotator (gold standard).                                 | 
| retweet_count                | Numerical     | Number of times the tweet was retweeted.                                                    | 
| text                         | Text          | The full content of the tweet.                                                              | 
| tweet_coord                  | Geospatial    | Latitude and longitude coordinates where the tweet was posted, if available.                | 
| tweet_created                | Datetime      | Timestamp when the tweet was created.                                                       | 
| tweet_location               | Text          | Location specified in the user's profile.                                                   | 
| user_timezone                | Categorical   | Time zone specified in the user's profile.                                                  | 

<br/>

## Data Used for Modeling

| Feature                      | Data Type   | Description  |
|-----------------------------|-------------|--------------|
| **Target Variable: `encoded_sentiment`** | Categorical | This is an engineered variable derived from `airline_sentiment` for multi-class sentiment classification. It encodes sentiment as: 0 = Negative, 1 = Neutral, 2 = Positive. |
| **Feature: `text`**         | Text        | Contains consumer tweets about U.S. airlines. This field undergoes preprocessing, including removal of URLs and mentions (`@`), and stopword removal |


<br/>

# 1. Load Data

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../../data/tweets.csv.gz", compression="gzip")

In [2]:
# Show basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [3]:
# Show the first few rows
df.head(3)

tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                        0.0  Virgin America   
2            NaN                        NaN  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnardino                 NaN              0   
2                    NaN  yvonnalynn                 NaN              0   

                                                text tweet_coord  \
0                @VirginAmerica What @dhepburn said.         NaN   
1  @VirginAmerica plus you've added commercials t...         NaN   
2  @VirginAmerica I didn't today... Must mean I n...         NaN   

               tweet_created tweet_location               user_timezone  
0  2015-02-24 11:35:52 -0800            NaN  Eastern Time (US & Canada)  
1  2015-02-24 11:15:59 -0800            NaN  Pacific Time (US & Canada)  
2  2015-02-24 11:15:48 -0800      Lets Play  Central Time (US & Canada)

# 2. Data Preprocessing

## 2.1 Handle Duplicates

In [4]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

Number of duplicate rows: 36


In [5]:
# Confirm the shape after removal
print(f"Shape after dropping duplicates: {df.shape}")

Shape after dropping duplicates: (14604, 15)


## 2.2 Handle Missing Values

In [6]:
# Check for missing values for each variables in the dataset
print("\nMissing values count for each variables:")
print("-------------------------------------------")
print(df.isnull().sum())

print("""\n\n**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values""")


Missing values count for each variables:
-------------------------------------------
tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5445
negativereason_confidence        4101
airline                             0
airline_sentiment_gold          14564
name                                0
negativereason_gold             14572
retweet_count                       0
text                                0
tweet_coord                     13589
tweet_created                       0
tweet_location                   4723
user_timezone                    4814
dtype: int64


**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values


## 2.3 Text Processing

### 2.3.1 Feature Engineering

For feature engineering in sentiment analysis, we will perform the following steps:
- `Clean text`: lowercase the twitter text, and remove urls & mentions from the twitter text

In [7]:
import re

# Step 1: Lowercase and clean the text
def clean_text(text):
    text = text.lower()                                 # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text)                    # Remove mentions
    # text = re.sub(r'#\w+', '', text)                    # NOTE: Do not remove hashtags, 
                                                                # as there is a lot of hashtags with sentiment indication, 
                                                                # such as '#thankyou', '#happycustomer', etc...
    # text = re.sub(r'[^a-z\s]', '', text)                # NOTE: no need to remove numbers and punctuation for llm embedding
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Copy only the 'text' column to df_copy
df2 = df[['text', 'airline_sentiment']].copy()

# processed text after rule-based removing URL, twitter username
df2['clean_text'] = df2['text'].apply(clean_text)



In [8]:
df2.shape

(14604, 3)

In [9]:
# Check for missing values per column
print("\nMissing values per column:")
print(df2.isnull().sum())


Missing values per column:
text                 0
airline_sentiment    0
clean_text           0
dtype: int64


### 2.3.2 Target Engineering

We will convert the 'airline_sentiment' column into numerical values to use it as the target variable in our model, where `negative` = 0, `neutral` = 1, and `positive` = 2 

In [10]:
print("The target variable contains unique values of: ", df2['airline_sentiment'].unique(), 
      "which we are going to map it into 0, 1 and 2 respectively")

# Encode the sentiment column
df2['encoded_sentiment'] = df2['airline_sentiment'].map({"negative": 0, "neutral": 1, "positive": 2})


The target variable contains unique values of:  ['neutral' 'positive' 'negative'] which we are going to map it into 0, 1 and 2 respectively


## 3.0 Data Preparation for Modeling

In [11]:
df2

text airline_sentiment  \
0                    @VirginAmerica What @dhepburn said.           neutral   
1      @VirginAmerica plus you've added commercials t...          positive   
2      @VirginAmerica I didn't today... Must mean I n...           neutral   
3      @VirginAmerica it's really aggressive to blast...          negative   
4      @VirginAmerica and it's a really big bad thing...          negative   
...                                                  ...               ...   
14635  @AmericanAir thank you we got on a different f...          positive   
14636  @AmericanAir leaving over 20 minutes Late Flig...          negative   
14637  @AmericanAir Please bring American Airlines to...           neutral   
14638  @AmericanAir you have my money, you change my ...          negative   
14639  @AmericanAir we have 8 ppl so we need 2 know h...           neutral   

                                              clean_text  encoded_sentiment  
0                                             what said.                  1  
1      plus you've added commercials to the experienc...                  2  
2      i didn't today... must mean i need to take ano...                  1  
3      it's really aggressive to blast obnoxious "ent...                  0  
4               and it's a really big bad thing about it                  0  
...                                                  ...                ...  
14635  thank you we got on a different flight to chic...                  2  
14636  leaving over 20 minutes late flight. no warnin...                  0  
14637    please bring american airlines to #blackberry10                  1  
14638  you have my money, you change my flight, and d...                  0  
14639  we have 8 ppl so we need 2 know how many seats...                  1  

[14604 rows x 4 columns]

In [12]:
# Split into features (X) and target labels (y)
X = df2['clean_text']
y = df2['encoded_sentiment']

### 3.1 Train test validation split with stratified sampling

In [13]:
from sklearn.model_selection import train_test_split

# Train test validation split with ratio of 70:15:15
X_train, X_temp, y_train, y_temp = train_test_split(X, y, 
                                        stratify=y,            
                                        train_size=0.7, 
                                        random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, 
                                        stratify=y_temp,            
                                        train_size=0.5, 
                                        random_state=42)

In [14]:
# Export train and test dataset to `data` folder
pd.concat([X_train, y_train], axis=1).to_csv("../../data/train.csv.gz", index=False)
pd.concat([X_valid, y_valid], axis=1).to_csv("../../data/valid.csv.gz", index=False)
pd.concat([X_test, y_test], axis=1).to_csv("../../data/test.csv.gz", index=False)

In [15]:
# preview the training set
pd.read_csv("../../data/train.csv.gz", compression="gzip", index_col=False)

clean_text  encoded_sentiment
0      was wondering if you guys recieved my dm and w...                  1
1      you guys get rid of the hip hop stations on si...                  1
2      ok it's now been 7 months waiting to hear from...                  0
3                      it was last night's 1235/ord-lga.                  1
4      many thanks, as always your employees are prof...                  1
...                                                  ...                ...
10217  how many others didn't know you are stopping n...                  0
10218                                 party in #hotlanta                  1
10219  epic fail @ clt ystrdy w/cancelled flightled f...                  0
10220  random q: what's the distribution of elevate a...                  1
10221                        did you get your 757 model?                  1

[10222 rows x 2 columns]

In [16]:
# Verify that class distribution is preserved after the train-test split (i.e., stratified correctly)

print("Train class distribution:")
print(y_train.value_counts(normalize=True))

print("Valid class distribution:")
print(y_valid.value_counts(normalize=True))

print("\nTest class distribution:")
print(y_test.value_counts(normalize=True))

print("""\n**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.""")

Train class distribution:
encoded_sentiment
0    0.627177
1    0.211602
2    0.161221
Name: proportion, dtype: float64
Valid class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

Test class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.


### 3.2 Text Representation with LLM embedding

In [17]:
import os
import pandas as pd 
from utils.prepare_llm_embedding import generate_embeddings_from_series 

EMBEDDING_TRAIN = "../../data/llm_embedding_train.csv.gz"
if os.path.exists(EMBEDDING_TRAIN):
    pass
else:
    processed_text_series = pd.Series(X_train.to_list(),
                                    index=X_train.index.to_list()) 
    llm_embedding_train = generate_embeddings_from_series(processed_text_series,
                            additional_data={"encoded_sentiment": y_train.to_list()},
                            output_csv_path="../../data/llm_embedding_train.csv.gz",
                            max_workers=20)
    print(llm_embedding_train)

In [18]:
import pandas as pd 
from utils.prepare_llm_embedding import generate_embeddings_from_series 

EMBEDDING_VALID = "../../data/llm_embedding_valid.csv.gz"
if os.path.exists(EMBEDDING_VALID):
    pass
else:
    processed_text_series = pd.Series(X_valid.to_list(), 
                                    index=X_valid.index.to_list()) 
    llm_embedding_valid = generate_embeddings_from_series(processed_text_series,
                            additional_data={"encoded_sentiment": y_valid.to_list()},
                            output_csv_path="../../data/llm_embedding_valid.csv.gz",
                            max_workers=20)
    print(llm_embedding_valid)

In [19]:
import pandas as pd 
from utils.prepare_llm_embedding import generate_embeddings_from_series 

EMBEDDING_TEST = "../../data/llm_embedding_test.csv.gz"
if os.path.exists(EMBEDDING_TEST):
    pass
else:
    processed_text_series = pd.Series(X_test.to_list(), 
                                    index=X_test.index.to_list()) 
    llm_embedding_test = generate_embeddings_from_series(processed_text_series,
                            additional_data={"encoded_sentiment": y_test.to_list()},
                            output_csv_path="../../data/llm_embedding_test.csv.gz",
                            max_workers=20)
    print(llm_embedding_test)

In [20]:
import numpy
import json

train_vectorized = pd.read_csv("../../data/llm_embedding_train.csv.gz", compression="gzip")
valid_vectorized = pd.read_csv("../../data/llm_embedding_valid.csv.gz", compression="gzip")
test_vectorized = pd.read_csv("../../data/llm_embedding_test.csv.gz", compression="gzip")

X_train_vectorized = train_vectorized["embedding_json"].apply(json.loads) # convert string into a list of 765 items in 1 column
X_train_vectorized = numpy.vstack(X_train_vectorized) # turn that list of 765 items into 765 features / columns
y_train = train_vectorized["encoded_sentiment"]

X_valid_vectorized = valid_vectorized["embedding_json"].apply(json.loads)
X_valid_vectorized = numpy.vstack(X_valid_vectorized)

X_test_vectorized = test_vectorized["embedding_json"].apply(json.loads)
X_test_vectorized = numpy.vstack(X_test_vectorized)

y_valid = valid_vectorized["encoded_sentiment"]
y_test = test_vectorized["encoded_sentiment"]

### 3.3 Handling class imbalance issue with SMOTE

In [21]:
from imblearn.over_sampling import SMOTE

# Handling imbalanced using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

In [22]:
# Check class distribution before and after applying SMOTE to confirm successful balancing

print("Class distribution in training set (before SMOTE):")
print(y_train.value_counts())

print("\nClass distribution in training set (after SMOTE):")
print(y_train_resampled.value_counts())

print("""\n**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.""")


Class distribution in training set (before SMOTE):
encoded_sentiment
0    6411
1    2163
2    1648
Name: count, dtype: int64

Class distribution in training set (after SMOTE):
encoded_sentiment
0    6411
1    6411
2    6411
Name: count, dtype: int64

**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.


## 4.0 Modeling

(i) Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
lr_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and valid sets
y_train_pred_lr = lr_model.predict(X_train_resampled)
y_valid_pred_lr = lr_model.predict(X_valid_vectorized)
y_test_pred_lr = lr_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_lr)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_lr)
test_accuracy = accuracy_score(y_test, y_test_pred_lr)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_lr, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_lr, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_lr, average='weighted')
test_precision = precision_score(y_test, y_test_pred_lr, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_lr, average='weighted')
test_recall = recall_score(y_test, y_test_pred_lr, average='weighted')


# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Validation F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Validation Precision:  {valid_precision:.4f}")
print(f"Test F1-score:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Validation Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_lr))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_lr))

Train Accuracy: 0.8874
Validation Accuracy:  0.8389
Test Accuracy:  0.8476

Train F1-score: 0.8875
Validation F1-score:  0.8441
Test F1-score:  0.8521

Train Precision: 0.8876
Validation Precision:  0.8559
Test F1-score:  0.8621

Train Recall: 0.8874
Validation Recall:  0.8389
Test Recall:  0.8476

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.95      0.86      0.91      1374
           1       0.66      0.80      0.72       464
           2       0.78      0.85      0.81       353

    accuracy                           0.85      2191
   macro avg       0.80      0.84      0.81      2191
weighted avg       0.86      0.85      0.85      2191

Confusion Matrix (Test):
[[1188  145   41]
 [  51  369   44]
 [  11   42  300]]


(ii) Decision Tree

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Decision Tree
dt_model = DecisionTreeClassifier(max_depth=100, 
                                  min_samples_split=10, 
                                  criterion='entropy', 
                                  min_samples_leaf=100,
                                  random_state=42)

# Train the model
dt_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and test sets
y_train_pred_dt = dt_model.predict(X_train_resampled)
y_valid_pred_dt = dt_model.predict(X_valid_vectorized)
y_test_pred_dt = dt_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_dt)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_dt)
test_accuracy = accuracy_score(y_test, y_test_pred_dt)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_dt, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_dt, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_dt, average='weighted')
test_precision = precision_score(y_test, y_test_pred_dt, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_dt, average='weighted')
test_recall = recall_score(y_test, y_test_pred_dt, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_dt))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_dt))

Train Accuracy: 0.7899
Valid Accuracy:  0.7175
Test Accuracy:  0.7111

Train F1-score: 0.7903
Valid F1-score:  0.7315
Test F1-score:  0.7241

Train Precision: 0.7909
Valid Precision:  0.7616
Valid Precision:  0.7616

Train Recall: 0.7899
Valid Recall:  0.7175
Test Recall:  0.7111

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.89      0.75      0.81      1374
           1       0.45      0.61      0.52       464
           2       0.61      0.68      0.64       353

    accuracy                           0.71      2191
   macro avg       0.65      0.68      0.66      2191
weighted avg       0.75      0.71      0.72      2191

Confusion Matrix (Test):
[[1035  265   74]
 [ 104  284   76]
 [  28   86  239]]


(iii) XGBoost

In [ ]:
import os
import joblib
import xgboost as xgb
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the XGBoost Classifier
model_path = "../../models/llm_embedding_xgb_model.pkl"
if os.path.exists(model_path):
    print("Loading existing model...")
    xgb_model = joblib.load(model_path)
else:
    # Train the model
    xgb_model = xgb.XGBClassifier(max_depth=10,
                                random_state=42,
                                # Introduce randomness to make training faster and reduce overfitting
                                subsample=0.8, ## Uses 80% of the data for each tree.
                                colsample_bytree=0.8, ## Uses 80% of the features for each tree.
                                # the parameters below make the model trained faster by enabling parallelism
                                n_jobs = -1)
    xgb_model.fit(X_train_resampled, y_train_resampled)
    # Export and load previously trained model to avoid retraining every time
    joblib.dump(xgb_model, model_path)
    print("Model saved.")

# Predictions on training and test sets
y_train_pred_xgb = xgb_model.predict(X_train_resampled)
y_valid_pred_xgb = xgb_model.predict(X_valid_vectorized)
y_test_pred_xgb = xgb_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_xgb)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_xgb)
test_accuracy = accuracy_score(y_test, y_test_pred_xgb)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_xgb, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_xgb, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_xgb, average='weighted')
test_precision = precision_score(y_test, y_test_pred_xgb, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_xgb, average='weighted')
test_recall = recall_score(y_test, y_test_pred_xgb, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Test Precision:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_xgb))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_xgb))

Model saved.
Train Accuracy: 0.9985
Valid Accuracy:  0.8489
Test Accuracy:  0.8498

Train F1-score: 0.9985
Valid F1-score:  0.8502
Test F1-score:  0.8507

Train Precision: 0.9985
Valid Precision:  0.8519
Test Precision:  0.8518

Train Recall: 0.9985
Valid Recall:  0.8489
Test Recall:  0.8498

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1374
           1       0.70      0.72      0.71       464
           2       0.79      0.81      0.80       353

    accuracy                           0.85      2191
   macro avg       0.80      0.81      0.81      2191
weighted avg       0.85      0.85      0.85      2191

Confusion Matrix (Test):
[[1241   98   35]
 [  86  336   42]
 [  24   44  285]]
